In [1]:
import os, sys
import uproot, awkward
import ROOT as rt
import numpy as np
from array import array
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

Welcome to JupyROOT 6.24/06


In [2]:
#inFile_k100 = "/Users/shubhampandey/work/geant4/k100sim_anthony/sim_files/sim_2M_neutrons/sim_2M_events_PuBe.root"
#inFile_k100 = "/Users/shubhampandey/work/geant4/k100sim_anthony/sim_files/sim_NaI_51M/sim_NaI_51M.root"
inFile_k100 = "/Users/shubhampandey/work/geant4/k100sim_anthony/sim_files/sim_100k_nCap_cylinder/sim_100k_nCap_cylinder.root"
#inFile_k100 = "/Users/shubhampandey/work/geant4/k100sim_anthony/sim_files/sim_50M_PuBe_sourceAndshields/sim_50M_PuBe_sourceAndshields_0001.root"
#inFile_k100 = "/Users/shubhampandey/work/geant4/k100sim_anthony/sim_files/sim_50M_PuBe_sourceAndshields_boron_shields/sim_50M_PuBe_sourceAndshields_boronShield_V2H2.root"
file_k100 = uproot.open(inFile_k100)
if(not file_k100):
    print("could not open file: %s"%(inFile_k100))
    sys.exit(0)
    
tree_k100 = file_k100["simtree"]

if(not tree_k100):
    print("simtree does not exist in file: %s"%(file_k100))
    sys.exit(0)

save_ = False
textOnTop = rt.TLatex()
textOnTop.SetTextSize(0.05)

In [3]:
# From k100sim
EV = tree_k100["EV"].array(library="np")
P = tree_k100["P"].array(library="np")
Type = tree_k100["Type"].array(library="np")
DT = tree_k100["DT"].array(library="np")
TS = tree_k100["TS"].array(library="np")
E1 = tree_k100["E1"].array(library="np")
D3 = tree_k100["D3"].array(library="np")
X1 = tree_k100["X1"].array(library="np")
Y1 = tree_k100["Y1"].array(library="np")
Z1 = tree_k100["Z1"].array(library="np")
X3 = tree_k100["X3"].array(library="np")
Y3 = tree_k100["Y3"].array(library="np")
Z3 = tree_k100["Z3"].array(library="np")
time1 = tree_k100["time1"].array(library="np")
time3 = tree_k100["time3"].array(library="np")
nCap = tree_k100["nCap"].array(library="np")

# canvas
c = rt.TCanvas("c","c",800,600)

In [4]:
h_k100sim_Nphotons_all = rt.TH1F("h_Nphotons_all","Number of photons from k100Sim",50,0,50)
h_pdgid_parent = rt.TH1F("h_pdgid","PDGID of parent particle",3000,0,3000)

h_parent_energy_log10 = rt.TH1F("h_parent_energy_log10","Energy of neutrons",150,-10,5)
h_parent_energy_log10.GetXaxis().SetTitle("log10(Parent Energy [MeV])")

h_parent_energy_log10_inSi = rt.TH1F("h_parent_energy_log10_inSi","Energy of neutrons traversing silicon",150,-10,5)
h_parent_energy_log10_inSi.GetXaxis().SetTitle("log10(Parent Energy [MeV])")

h_parent_energy_log10_on_nCap = rt.TH1F("h_parent_energy_log10_on_nCap","Energy of neutrons on nCap",150,-10,5)
h_parent_energy_log10_on_nCap.GetXaxis().SetTitle("log10(Parent Energy [MeV])")

h_energy_neutron_nCap = rt.TH1F("h_energy_neutron_nCap","Energy of neutrons for nCap",150,0,1.5)
h_energy_neutron_nCap.GetXaxis().SetTitle("Energy [eV]")

h_Edep_neutron = rt.TH1F("h_Edep_neutron","Energy deposited by neutrons before nCap",150,-10,5)
h_Edep_neutron.GetXaxis().SetTitle("log10(Neutron Edep [MeV])")

h_neutron_x_y = rt.TH2F("h_neutron_x_y","Position of neutrons registered in detector",120,-60,60,120,-60,60)
h_neutron_x_y.GetXaxis().SetTitle("x [mm]")
h_neutron_x_y.GetYaxis().SetTitle("y [mm]")



h_neutron_x_y_nCap = rt.TH2F("h_neutron_x_y_nCap","Position of neutron captures registered in Si crystal",120,-60,60,120,-60,60)
h_neutron_x_y_nCap.GetXaxis().SetTitle("x [mm]")
h_neutron_x_y_nCap.GetYaxis().SetTitle("y [mm]")

h_neutron_x_y_z_nCap = rt.TH3F("h_neutron_x_y_z_nCap","Position of neutron captures registered in Si crystal",120,-60,60,120,-60,60,60,10.0, 70.0)
h_neutron_x_y_z_nCap.GetXaxis().SetTitle("x [mm]")
h_neutron_x_y_z_nCap.GetYaxis().SetTitle("y [mm]")
h_neutron_x_y_z_nCap.GetYaxis().SetTitle("z [mm]")


h_neutron_x = rt.TH1F("h_neutron_x","X position of neutrons registered in the detector",120,-60,60)
h_neutron_x.GetXaxis().SetTitle("x [mm]")

h_neutron_y = rt.TH1F("h_neutron_y","y position of neutrons registered in the detector",120,-60,60)
h_neutron_y.GetXaxis().SetTitle("y [mm]")

h_neutron_z = rt.TH1F("h_neutron_z","Z position of neutrons registered in the detector",600, 10.0, 70.0)
h_neutron_z.GetXaxis().SetTitle("z [mm]")

h_neutron_dR_nCap = rt.TH1F("h_neutron_dR_nCap","dR(preSetp, postStep) for neutron capture",200, 0.0, 200.0)
h_neutron_dR_nCap.GetXaxis().SetTitle("dR [mm]")

h_photon_energy_nCapInNaI = rt.TH1F("h_photon_energy_nCapInNaI","Energy of photons produced on nCap in NaI as per Geant4",150,0,15)
h_photon_energy_nCapInNaI.GetXaxis().SetTitle("Energy [MeV]")

h_energy_gamma_nCap_Si = rt.TH1F("h_energy_gamma_nCap_Si","Energy of gammas on nCap Si",500,0,10)
h_energy_gamma_nCap_Si.GetXaxis().SetTitle("Energy [MeV]")

h_energySum_gamma_nCap_Si = rt.TH1F("h_energySum_gamma_nCap_Si","EnergySum of gammas on nCap Si",1000,0,50)
h_energySum_gamma_nCap_Si.GetXaxis().SetTitle("Energy [MeV]")

h_nGammas_nCap_Si = rt.TH1F("h_nGammas_nCap_Si","Nb of gammas on nCap in Si",20,0,20)
h_nGammas_nCap_Si.GetXaxis().SetTitle("Nb of gammas")


total_capture_events = 0
capture_in_si = 0
capture_in_NaI = 0
events_with_no_neutron = []
events_with_neutron = []
events_with_nCap_in_NaI = []
events_with_nCap_in_Si = []
debug_count = 0
for i in tqdm(range(len(EV))):
    tracks = TS[i]/100000
    tracks = tracks.astype('int32')
    neutron_tracks = tracks[Type[i] == 2112]
    neutron_parents = P[i][Type[i] == 2112]
    
    #neutron_tracks = neutron_tracks[neutron_parents == 0]
    
    
    if(len(neutron_tracks) == 0): # No neutron in the event
        events_with_no_neutron.append(i)
        continue
        
    unique_tracks, indices = np.unique(neutron_tracks, return_index=True)
    
#     if(len(unique_tracks) !=1): # Sanity check
#         print("More than one unique tracks for event = ",i)
#         break
    
    events_with_neutron.append(i) 
    neutron_energy = E1[i][Type[i] == 2112]
    neutron_parents = P[i][Type[i] == 2112]
#     if(len(neutron_energy[neutron_parents == 0]) == 0):
#         continue
    neutron_DT = DT[i][Type[i] == 2112]
    neutron_D3 = D3[i][Type[i] == 2112]
    
    neutron_X1 = X1[i][Type[i] == 2112]
    neutron_Y1 = Y1[i][Type[i] == 2112]
    neutron_Z1 = Z1[i][Type[i] == 2112]
    neutron_X3 = X3[i][Type[i] == 2112]
    neutron_Y3 = Y3[i][Type[i] == 2112]
    neutron_Z3 = Z3[i][Type[i] == 2112]
    neutron_capture = nCap[i][Type[i] == 2112]
    neutron_tracks = TS[i][Type[i] == 2112]
    
    parent_neutron_en = (neutron_energy)[0]
    parent_neutron_X3 = (neutron_X3)[0]
    parent_neutron_Y3 = (neutron_Y3)[0]
    parent_neutron_Z3 = (neutron_Z3)[0]
    parent_neutron_DT = (neutron_DT)[0]
    
    h_parent_energy_log10.Fill(np.log10(parent_neutron_en)) # Use first neutron resgiterd in Si
    h_neutron_x_y.Fill(parent_neutron_X3,parent_neutron_Y3)
    h_neutron_x.Fill(parent_neutron_X3)
    h_neutron_y.Fill(parent_neutron_Y3)
    h_neutron_z.Fill(parent_neutron_Z3)
    
    if(parent_neutron_DT == 1):
        h_parent_energy_log10_inSi.Fill(np.log10(parent_neutron_en))
    
    nCap_Si_neutron_track = -1
    nCap_Si = False
    for i_ncap in (np.where(neutron_capture == 1))[0]:
        Edep_ncap = neutron_D3[i_ncap]
        if(Edep_ncap > 0.):
            print("non zero Edep %f on nCap by neutron for event = %d"%(Edep_ncap,i))
#         if(np.sum(neutron_D3) > )
        total_capture_events += 1
#         h_parent_energy_log10_on_nCap.Fill(np.log10(neutron_energy[i_ncap]))
#         h_energy_neutron_nCap.Fill(neutron_energy[i_ncap]*1.e6)
        dx = neutron_X3[i_ncap] - neutron_X1[i_ncap]
        dy = neutron_Y3[i_ncap] - neutron_Y1[i_ncap]
        dz = neutron_Z3[i_ncap] - neutron_Z1[i_ncap]
        h_Edep_neutron.Fill(np.sum(neutron_D3))
        h_neutron_dR_nCap.Fill(np.sqrt(dx*dx + dy*dy + dz*dz))
        if(neutron_DT[i_ncap] == 1):
            capture_in_si += 1
            h_neutron_x_y_nCap.Fill(neutron_X3[i_ncap],neutron_Y3[i_ncap])
            h_neutron_x_y_z_nCap.Fill(neutron_X3[i_ncap],neutron_Y3[i_ncap],neutron_Z3[i_ncap])
            h_parent_energy_log10_on_nCap.Fill(np.log10(neutron_energy[i_ncap]))
            h_energy_neutron_nCap.Fill(neutron_energy[i_ncap]*1.e6)
            events_with_nCap_in_Si.append(i)
            nCap_Si = True
            nCap_Si_neutron_track = int(neutron_tracks[i_ncap]/100000)*100000
        else:
            capture_in_NaI += 1
            events_with_nCap_in_NaI.append(i)
    
    if(nCap_Si):
        esum_gammas = 0.
        n_gammas = 0
        
        for j,capture in enumerate(nCap[i]):
            track_temp = int(TS[i][j]/100000)*100000
            step_temp = TS[i][j] - (track_temp)
            if(P[i][j] == nCap_Si_neutron_track and step_temp == 1):
                if(Type[i][j] == 22):
                    h_energy_gamma_nCap_Si.Fill(E1[i][j])
                    esum_gammas += E1[i][j]
                    n_gammas += 1
        
        h_nGammas_nCap_Si.Fill(n_gammas)
        h_energySum_gamma_nCap_Si.Fill(esum_gammas)
        if(debug_count < 10 and n_gammas >= 9):
            print("%d photons, %f MeV Edep for event %d"%(n_gammas, esum_gammas, i))
            debug_count += 1
    
                
    
        
print ("Total events = %d"%(len(EV)))
print ("Events with no registered neutrons in detector = %d"%(len(events_with_no_neutron)))
print ("Events with registered neutrons in NaI+Si = %d"%(len(events_with_neutron)))
#print (events_with_no_neutron)
print ("Neutrons registered in Silicon = %d"%(h_parent_energy_log10_inSi.GetEntries()))
print ("Neutrons registered in NaI = %d"%(len(events_with_neutron) - h_parent_energy_log10_inSi.GetEntries()))
print ("Total neutron capture events = %d out of %d events."%(total_capture_events,len(events_with_neutron)))
print ("Total neutron capture events in Si = %d out of %d events."%(capture_in_si,total_capture_events))
print ("Total neutron capture events in NaI = %d out of %d events."%(capture_in_NaI,total_capture_events))


#     for j,parent in enumerate(P[i]):
#         if(parent == 0):
#             h_pdgid_parent.Fill(Type[i][j])
#             h_parent_energy_log10.Fill(np.log10(E1[i][j]))
#             h_neutron_x_y.Fill(X1[i][j],Y1[i][j])
#             h_neutron_x.Fill(X1[i][j])
#             h_neutron_y.Fill(Y1[i][j])
#             h_neutron_z.Fill(Z1[i][j])
#             if(nCap[i][j] == 1):
#                 h_parent_energy_log10_on_nCap.Fill(np.log10(E1[i][j]))
#                 h_energy_neutron_nCap.Fill(E1[i][j]*1.e6)

#                 #print (E1[i][j]*1.e6)

  4%|█████▍                                                                                                                   | 4465/99928 [00:00<00:04, 22589.05it/s]

10 photons, 19.227083 MeV Edep for event 227
12 photons, 35.759128 MeV Edep for event 281
9 photons, 20.194300 MeV Edep for event 284
9 photons, 24.022582 MeV Edep for event 310
9 photons, 15.684771 MeV Edep for event 365
9 photons, 28.851979 MeV Edep for event 467
9 photons, 23.919011 MeV Edep for event 526
12 photons, 35.873767 MeV Edep for event 670
9 photons, 23.181782 MeV Edep for event 826
11 photons, 17.920479 MeV Edep for event 1139


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99928/99928 [00:04<00:00, 23553.73it/s]


Total events = 99928
Events with no registered neutrons in detector = 96
Events with registered neutrons in NaI+Si = 99832
Neutrons registered in Silicon = 98726
Neutrons registered in NaI = 1106
Total neutron capture events = 23097 out of 99832 events.
Total neutron capture events in Si = 22417 out of 23097 events.
Total neutron capture events in NaI = 680 out of 23097 events.


In [5]:
%jsroot on
rt.gPad.SetLogy(0)
h_neutron_x_y_z_nCap.Draw("box")
c.Draw()
if(save_):
    c.SaveAs("neutron_capture_si_3D_100k.png")

In [6]:
%jsroot on
h_parent_energy_log10_inSi.Draw()
h_parent_energy_log10_on_nCap.SetLineColor(rt.kRed)
h_parent_energy_log10_on_nCap.SetLineWidth(2)
h_parent_energy_log10_on_nCap.SetLineStyle(2)
h_parent_energy_log10_on_nCap.Draw("sames")
rt.gPad.SetLogy()
c.Draw()
if(save_):
    c.SaveAs("EnergyNeutronOverlay_100k.png")

In [9]:
h_nGammas_nCap_Si.Draw()
rt.gPad.SetLogy(0)

c.Draw()

In [10]:
h_energy_gamma_nCap_Si.Draw()
rt.gPad.SetLogy(0)
c.Draw()


In [12]:
h_energySum_gamma_nCap_Si.Draw()
rt.gPad.SetLogy(0)
c.Draw()